In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split

import unicodedata
import re
import numpy as np
import os
import io
import time
import collections
import json
import string

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

In [3]:
#def dictionary

In [4]:
def tokenize(line,token='word'):
    if token == 'word':
        return [line.split(' ')]
    elif token == 'char':
        return [list(line)]
    else:
        print('ERROR: unknown token type '+token)

In [5]:
def count_tokens(tokanized_sentences):
    # Flatten a list of token lists into a list of tokens
    tokens = [tk for line in tokanized_sentences for tk in line]
    return collections.Counter(tokens)

In [6]:
filename = 'MLDS_hw2_1_data/training_label.json'
with open(filename, 'r') as f:
    datastore = json.load(f)

In [7]:
vid_feat_set = {}
vid_sentence_set = {}

for data in datastore:
    video_id = data["id"]
    vid_feat_set[video_id]=None
    vid_sentence_set[video_id]=None

In [8]:
for data in datastore:
    video_id = data["id"]
    #print("reading features in: %s" % video_id)
    features = np.load("MLDS_hw2_1_data/training_data/feat/{}.npy".format(video_id))
    #print(video_id)
    
    vid_framefeats = []

    for array in features:
        vid_framefeats.append(array)
    
    vid_feat_set[video_id] = vid_framefeats
    
    #print("reading sentences in: %s" % video_id)
    sentences = data["caption"]
    sentences = [word.lower() for word in sentences] #Normalize the case
    table = str.maketrans('', '', string.punctuation) #Normalize the punctuation
    sentences = [word.translate(table) for word in sentences]
    vid_sentence_set[video_id] = sentences

In [9]:
#sentence_set
print("The number of videos in the training set are %d and each video has 80 frames with 4096 features/units each" % len(vid_feat_set))

The number of videos in the training set are 1450 and each video has 80 frames with 4096 features/units each


In [10]:
# for set_i in vid_sentence_set:
#     print(set_i)
#     print(vid_sentence_set[set_i])


In [11]:
# #vid_feat_set['WqQonRVs7WA_0_10.avi']

# count = 0
# for x in vid_sentence_set: 
#     if isinstance(vid_sentence_set[x], list): 
#         count += len(vid_sentence_set[x]) 
# print(count) 

In [12]:
# Mapping string tokens to numertical indices.
def listVocab(vid_sentence_set):
    
    PAD_token = 0
    BOS_token = 1
    EOS_token = 2
    UNK_token = 3
    
    all_tokens = []
    values = [0,1,2,3]
    token_index = [(PAD_token, "<PAD>"), (BOS_token, "<BOS>"), (EOS_token, "<EOS>"), (UNK_token, "<UNK>")]
    for set_i in vid_sentence_set:
        sentence_set = vid_sentence_set[set_i]
        for line in sentence_set: 
            tokenized_captions = tokenize(line) #Seperate the words
            all_tokens += tokenized_captions
    
    counter = count_tokens(all_tokens) #Count the word repeatitions in each set
    
    counter_dict = counter.items()
    counter_sort = sorted(counter_dict, key=lambda x:x[1],reverse=True) #sort by frequency of occurance 
    #print(counter_sort)

    i = len(token_index)
    for token, freq in counter_sort:
        token_index.append((i,token))
        values+=[i]
        i+=1
    
    return [values, token_index, len(token_index)]

In [13]:
values, token_index, nums = listVocab(vid_sentence_set)
print("There are %d unique words in the captions dataset" % nums)

# integer encode
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(values)
print(integer_encoded)

# binary encode
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)


There are 6061 unique words in the captions dataset
[   0    1    2 ... 6058 6059 6060]


In [14]:
#counter_dict

NameError: name 'counter_dict' is not defined

In [ ]:
#np.shape(onehot_encoded)

In [14]:
def flattenList(nestedList,output): 
    for i in nestedList: 
        if type(i) == list: 
            flattenList(i,output) 
        else: 
            output.append(i) 
            
    return output

In [15]:
MAX_WORDS = 50

def num_encode(test_sentence,token_index,tokenized_sentence=[],num_encoded_sentence=[],onehot_encoded_sentence=[]):
    
    tokenized_sentence.clear()
    num_encoded_sentence.clear()
    onehot_encoded_sentence.clear()
    
    tokenized_sentence = ["<BOS>"] + tokenize(test_sentence) + ["<EOS>"]
    #print(tokenized_sentence)
    output=[]
    tokenized_sentence = flattenList(tokenized_sentence,output)

    while len(tokenized_sentence) < MAX_WORDS:
        tokenized_sentence.append("<PAD>")
    #print(len(tokenized_sentence))
   
    for token in tokenized_sentence:
        for i in range(0,len(token_index)):
            if token == token_index[i][1]: 
                num_encoded_sentence.append(i) 
                onehot_encoded_sentence.append(onehot_encoded[i])
        
    return tokenized_sentence, num_encoded_sentence, onehot_encoded_sentence


In [16]:
#for set_i in sentence_set:
num_encoded_dict = {}
onehot_encoded_dict = {}

for data in datastore:
    video_id = data["id"]
    num_encoded_dict[video_id]=None
    onehot_encoded_dict[video_id]=None
  
    
for vid in vid_sentence_set:
    sentence_set = vid_sentence_set[vid]
    
    num_encoded_per_set = []
    onehot_encoded_per_set = []

    for line in sentence_set:
        #print(line)
        _,encoded_sentence,onehot_encoded_sentence = num_encode(line,token_index)
        #print(len(tokenized_sen))
        encoded_sentence = list(encoded_sentence)
        onehot_encoded_sentence = list(onehot_encoded_sentence)

        #print(type(encoded_sentence)) 
        #print("nxt")
        num_encoded_per_set.append(encoded_sentence)
        #print(num_encoded_per_set)

        
        onehot_encoded_per_set.append(onehot_encoded_sentence)

    #print(num_encoded_per_set)
    
    num_encoded_dict[vid] = num_encoded_per_set
    onehot_encoded_dict[vid] = onehot_encoded_per_set

In [22]:
d={}
import sys
sys.getsizeof(d)

240

In [ ]:
# f = open("vid_feat_dict.txt","w")
# f.write(str(vid_framefeats))
# f.close()

In [ ]:
# f = open("vid_sentences_dict.txt","w")
# f.write(str(vid_sentence_set))
# f.close()

In [ ]:
#np.save('vid_feat_dict.npy',vid_feat_set)

In [ ]:
#np.save('vid_sentences_dict.npy',num_encoded_dict)

In [19]:
import pickle

with open('vid_feat_set.pickle', 'wb') as handle:
    pickle.dump(vid_feat_set, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [24]:
with open('num_encoded_dict.pickle', 'wb') as handle:
    pickle.dump(num_encoded_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [25]:
with open('onehot_encoded_dict.pickle', 'wb') as handle:
    pickle.dump(onehot_encoded_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

MemoryError: 

In [26]:
from sklearn.externals import joblib
filename = 'onehot_encoded_dict.sav'
joblib.dump(onehot_encoded_dict, filename)  

/home/adhitir/anaconda3/envs/tf_class/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


['onehot_encoded_dict.sav']